# Submit Training Job Demo

In [ ]:
import json
import time
import os
import requests

import fml_manager
from fml_manager import *

## For more details about the FMLManager, please refer to this [document](https://kubefate.readthedocs.io/README.html)

The DSL and Config file can be presented in JOSN format. 
Submitting a job with JSON defined in line, can use ```submit_job(self,dsl, config)```. 
Note: the parameters are dict, the JSON string have to transform to dict with ```json.loads```

In [ ]:
# dsl
secure_add_example = Component(name='secure_add_example_0',
                               module='SecureAddExample')

pipeline = Pipeline(
    secure_add_example
)

# Configuration
initiator = Initiator(role='guest',
                      party_id=9999)

job_parameters = JobParameters(work_mode=1)

role = RoleBuilder()\
    .add_guest(9999)\
    .add_host(9999).build()

secure_add_example_guest_config = {
    "seed": [
        123
    ]
}

secure_add_example_host_config = {
    "seed": [
        321
    ]
}
role_parameters = RoleParametersBuilder()\
    .add_host_module_config(module='secure_add_example_0', config=secure_add_example_host_config)\
    .add_guest_module_config(module='secure_add_example_0', config=secure_add_example_guest_config).build()

secure_add_example = {
    "partition": 10,
    "data_num": 1000
}

algorithm_parameters = AlgorithmParametersBuilder()\
    .add_module_config(module='secure_add_example_0', config=secure_add_example).build()

config = Config(
    initiator,
    job_parameters,
    role,
    role_parameters,
    algorithm_parameters
)

manager = fml_manager.FMLManager()
response = manager.submit_job(pipeline.to_dict(), config.to_dict())

In [ ]:
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
job_id = stdout['jobId']

In [ ]:
query_condition = QueryCondition(job_id)

job_status = manager.query_job(query_condition)
manager.prettify(job_status, verbose=True)

In [ ]:
manager.query_job_status(query_condition, max_tries=20)

We can also fetch the logs of job submitted, and save it to working folder.

In [ ]:
response = manager.fetch_job_log(job_id)